In [0]:
#Carga de la biblioteca
import pyspark
# Carga funciones extra
from pyspark.sql.functions import * 
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('primeros_pasos').getOrCreate()



#Instalación de bibliotecas adicionales
# Procesamiento del lenguaje natural
!pip install nltk
!pip install --upgrade pip
!python -m nltk.downloader all

!pip install sentiment-analysis-spanish
!pip install keras tensorflow

     |████████████████████████████████| 1.5 MB 5.6 MB/s 
     |████████████████████████████████| 76 kB 7.0 MB/s 
     |████████████████████████████████| 97 kB 10.4 MB/s 
     |████████████████████████████████| 764 kB 44.6 MB/s 
You should consider upgrading via the '/databricks/python3/bin/python -m pip install --upgrade pip' command.
     |████████████████████████████████| 2.1 MB 6.5 MB/s 
  Attempting uninstall: pip
    Found existing installation: pip 21.0.1
    Uninstalling pip-21.0.1:
      Successfully uninstalled pip-21.0.1
/usr/lib/python3.8/runpy.py:127: RuntimeWarning: 'nltk.downloader' found in sys.modules after import of package 'nltk', but prior to execution of 'nltk.downloader'; this may result in unpredictable behaviour
  warn(RuntimeWarning(msg))
[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/abc.zip.
[nltk_data]    | Downloading package alpino to /root/nltk_d

In [0]:
#Carga de NLTK
import nltk
import string
from sentiment_analysis_spanish import sentiment_analysis

In [0]:
#cargamos los datos como RDD
RDD_crudos = spark.sparkContext.textFile('/FileStore/tables/test.csv')

In [0]:
rdd_amazon = spark.sparkContext.textFile('/FileStore/tables/test.csv').map(lambda x: x.split('*'))


for opinion in rdd_amazon.take(5):
  print(opinion)

['"1","I got \'new\' tires from them and within two weeks got a flat. I took my car to a local mechanic to see if i could get the hole patched, but they said the reason I had a flat was because the previous patch had blown - WAIT, WHAT? I just got the tire and never needed to have it patched? This was supposed to be a new tire. \\nI took the tire over to Flynn\'s and they told me that someone punctured my tire, then tried to patch it. So there are resentful tire slashers? I find that very unlikely. After arguing with the guy and telling him that his logic was far fetched he said he\'d give me a new tire \\""this time\\"". \\nI will never go back to Flynn\'s b/c of the way this guy treated me and the simple fact that they gave me a used tire!"']
['"1","Don\'t waste your time.  We had two different people come to our house to give us estimates for a deck (one of them the OWNER).  Both times, we never heard from them.  Not a call, not the estimate, nothing."']
['"1","All I can say is the 

In [0]:
# Transformar textos a minúsculas
rdd_minus = rdd_amazon.map(lambda x: (x[0].lower()))

for letra in rdd_minus.take(5):
  print(letra)

"1","i got 'new' tires from them and within two weeks got a flat. i took my car to a local mechanic to see if i could get the hole patched, but they said the reason i had a flat was because the previous patch had blown - wait, what? i just got the tire and never needed to have it patched? this was supposed to be a new tire. \ni took the tire over to flynn's and they told me that someone punctured my tire, then tried to patch it. so there are resentful tire slashers? i find that very unlikely. after arguing with the guy and telling him that his logic was far fetched he said he'd give me a new tire \""this time\"". \ni will never go back to flynn's b/c of the way this guy treated me and the simple fact that they gave me a used tire!"
"1","don't waste your time.  we had two different people come to our house to give us estimates for a deck (one of them the owner).  both times, we never heard from them.  not a call, not the estimate, nothing."
"1","all i can say is the worst! we were the o

In [0]:
# Para poder tratar con el texto a nivel de palabras debemos hacer lo siguiente
rdd_palabras = rdd_minus.map(lambda x: (x.split(' ')))

for letra in rdd_palabras.take(5):
  print(letra)

['"1","i', 'got', "'new'", 'tires', 'from', 'them', 'and', 'within', 'two', 'weeks', 'got', 'a', 'flat.', 'i', 'took', 'my', 'car', 'to', 'a', 'local', 'mechanic', 'to', 'see', 'if', 'i', 'could', 'get', 'the', 'hole', 'patched,', 'but', 'they', 'said', 'the', 'reason', 'i', 'had', 'a', 'flat', 'was', 'because', 'the', 'previous', 'patch', 'had', 'blown', '-', 'wait,', 'what?', 'i', 'just', 'got', 'the', 'tire', 'and', 'never', 'needed', 'to', 'have', 'it', 'patched?', 'this', 'was', 'supposed', 'to', 'be', 'a', 'new', 'tire.', '\\ni', 'took', 'the', 'tire', 'over', 'to', "flynn's", 'and', 'they', 'told', 'me', 'that', 'someone', 'punctured', 'my', 'tire,', 'then', 'tried', 'to', 'patch', 'it.', 'so', 'there', 'are', 'resentful', 'tire', 'slashers?', 'i', 'find', 'that', 'very', 'unlikely.', 'after', 'arguing', 'with', 'the', 'guy', 'and', 'telling', 'him', 'that', 'his', 'logic', 'was', 'far', 'fetched', 'he', 'said', "he'd", 'give', 'me', 'a', 'new', 'tire', '\\""this', 'time\\"".', 

In [0]:
#Removemos stopwords (palabras vacias)
def removeStopWordsFunct(x):
    from nltk.corpus import stopwords
    stop_words=set(stopwords.words('english'))
    filteredSentence = [w for w in x if not w in stop_words]
    return filteredSentence

rdd_stop = rdd_palabras.map(removeStopWordsFunct)

for palabra in rdd_stop.take(10):
  print(palabra)

['"1","i', 'got', "'new'", 'tires', 'within', 'two', 'weeks', 'got', 'flat.', 'took', 'car', 'local', 'mechanic', 'see', 'could', 'get', 'hole', 'patched,', 'said', 'reason', 'flat', 'previous', 'patch', 'blown', '-', 'wait,', 'what?', 'got', 'tire', 'never', 'needed', 'patched?', 'supposed', 'new', 'tire.', '\\ni', 'took', 'tire', "flynn's", 'told', 'someone', 'punctured', 'tire,', 'tried', 'patch', 'it.', 'resentful', 'tire', 'slashers?', 'find', 'unlikely.', 'arguing', 'guy', 'telling', 'logic', 'far', 'fetched', 'said', "he'd", 'give', 'new', 'tire', '\\""this', 'time\\"".', '\\ni', 'never', 'go', 'back', "flynn's", 'b/c', 'way', 'guy', 'treated', 'simple', 'fact', 'gave', 'used', 'tire!"']
['"1","don\'t', 'waste', 'time.', '', 'two', 'different', 'people', 'come', 'house', 'give', 'us', 'estimates', 'deck', '(one', 'owner).', '', 'times,', 'never', 'heard', 'them.', '', 'call,', 'estimate,', 'nothing."']
['"1","all', 'say', 'worst!', '2', 'people', 'place', 'lunch,', 'place', 'fre

In [0]:
#Eliminamos signos de puntuación y espacios en blanco
import string
def removePunctuationsFunct(x):
    list_punct=list(string.punctuation)
    filtered = [''.join(c for c in s if c not in list_punct) for s in x] 
    filtered_space = [s for s in filtered if s] #remove empty space 
    return filtered

rdd_punct = rdd_stop.map(removePunctuationsFunct)

for palabra in rdd_punct.take(10):
  print(palabra)

['1i', 'got', 'new', 'tires', 'within', 'two', 'weeks', 'got', 'flat', 'took', 'car', 'local', 'mechanic', 'see', 'could', 'get', 'hole', 'patched', 'said', 'reason', 'flat', 'previous', 'patch', 'blown', '', 'wait', 'what', 'got', 'tire', 'never', 'needed', 'patched', 'supposed', 'new', 'tire', 'ni', 'took', 'tire', 'flynns', 'told', 'someone', 'punctured', 'tire', 'tried', 'patch', 'it', 'resentful', 'tire', 'slashers', 'find', 'unlikely', 'arguing', 'guy', 'telling', 'logic', 'far', 'fetched', 'said', 'hed', 'give', 'new', 'tire', 'this', 'time', 'ni', 'never', 'go', 'back', 'flynns', 'bc', 'way', 'guy', 'treated', 'simple', 'fact', 'gave', 'used', 'tire']
['1dont', 'waste', 'time', '', 'two', 'different', 'people', 'come', 'house', 'give', 'us', 'estimates', 'deck', 'one', 'owner', '', 'times', 'never', 'heard', 'them', '', 'call', 'estimate', 'nothing']
['1all', 'say', 'worst', '2', 'people', 'place', 'lunch', 'place', 'freezing', 'loaded', 'kids', 'toys', '2', 'bicycles', 'scoote

In [0]:
#Vamos a extrer la frecuencia de las palabras más usadas
freqDistRDD = rdd_punct.flatMap(lambda x : nltk.FreqDist(x).most_common()).map(lambda x: x).reduceByKey(lambda x,y : x+y).sortBy(lambda x: x[1], ascending = False)

freqDistRDD.take(10)

Out[18]: [('', 155951),
 ('food', 28458),
 ('place', 28074),
 ('good', 28071),
 ('like', 23485),
 ('get', 20858),
 ('one', 19911),
 ('would', 18347),
 ('time', 18314),
 ('service', 17550)]

In [0]:
#Análisis de sentimientos
def sentimentWordsFunct(x):
  from nltk.sentiment.vader import SentimentIntensityAnalyzer
  analyzer = SentimentIntensityAnalyzer() 
  senti_list_temp = []
  for i in x:
    y = ''.join(i) 
    vs = analyzer.polarity_scores(y)
    senti_list_temp.append((y, vs))
    senti_list_temp = [w for w in senti_list_temp if w]
  sentiment_list  = []
  for j in senti_list_temp:
    first = j[0]
    second = j[1]
    
    for (k,v) in second.items():
      if k == 'compound':
        if v < 0.0:
          sentiment_list.append((first, "Negative"))
        elif v == 0.0:
          sentiment_list.append((first, "Neutral"))
        else:
          sentiment_list.append((first, "Positive"))
  return sentiment_list

sentimentRDD = rdd_punct.map(sentimentWordsFunct)
sentimentRDD.take(10)

Out[20]: [[('1i', 'Neutral'),
  ('got', 'Neutral'),
  ('new', 'Neutral'),
  ('tires', 'Neutral'),
  ('within', 'Neutral'),
  ('two', 'Neutral'),
  ('weeks', 'Neutral'),
  ('got', 'Neutral'),
  ('flat', 'Neutral'),
  ('took', 'Neutral'),
  ('car', 'Neutral'),
  ('local', 'Neutral'),
  ('mechanic', 'Neutral'),
  ('see', 'Neutral'),
  ('could', 'Neutral'),
  ('get', 'Neutral'),
  ('hole', 'Neutral'),
  ('patched', 'Neutral'),
  ('said', 'Neutral'),
  ('reason', 'Neutral'),
  ('flat', 'Neutral'),
  ('previous', 'Neutral'),
  ('patch', 'Neutral'),
  ('blown', 'Neutral'),
  ('', 'Neutral'),
  ('wait', 'Neutral'),
  ('what', 'Neutral'),
  ('got', 'Neutral'),
  ('tire', 'Neutral'),
  ('never', 'Neutral'),
  ('needed', 'Neutral'),
  ('patched', 'Neutral'),
  ('supposed', 'Neutral'),
  ('new', 'Neutral'),
  ('tire', 'Neutral'),
  ('ni', 'Neutral'),
  ('took', 'Neutral'),
  ('tire', 'Neutral'),
  ('flynns', 'Neutral'),
  ('told', 'Neutral'),
  ('someone', 'Neutral'),
  ('punctured', 'Neutral'),
 